## Introduction to Tree Based Classifiers
#### Lewis Sears

By now, it should be clear that a machine learning algorithm to classify data begins with the same problem. Keeping notation consistent, we have a set of classes, $\chi = \{C_{1}, \ldots, C_{k})$, and we want to create a systematic way to classify some instance $\vec{x} = (x_1,\ldots, x_n)$ with vector elements that correspond to the $n$ features of our data set. 


### Decision Tree

For tree based classification algorithms, the simplest place to start is a decision tree. You should be able to tell by the picture below why they are called trees! We start with our data point $\vec{x}$, and from the top, each node represents a "question" about $\vec{x}$ that dictates the next node that we should send $\vec{x}$ to. at the end of this intense game of "20 questions" (maybe not exactly 20), we should be able to systematically decide what class $\vec{x}$ is in. The bottom nodes of the tree are called **leafs** and they are encoded with probabilities to determine the probability of classifying $\vec{x}$ based on pre-classified training data taking the same path down the tree. 
<img src="tree_images/DecisionTreePic.png" alt="drawing" width="550"/>
Imagine we have an animal and we want to classify whether that animal is a bird, a mammal, or a fish. We could ask simple questions to get to the bottom of it pretty quickly shown below: 
<img src="tree_images/simpledecisiontree.png" alt="drawing" width="550"/>

Your head should be screaming right now, **"But Lewis, how do we measure what questions are the most important and construct this???"** which is a valid question and the meat of the algorithm. Relax, we'll get to that. Before we do, the first part of our code will be how to create a question for specific features in the data set and to partition the data set based on the questions. We calculate how good a question is by its **Gini impurity**. To compute Gini impurity for a set of data with our $k$ classes in $\chi$, we sum the squared probabilities $p_{i}$, where $p_{i}$ refers to the probability of correctly classifying $A_{i}$ correctly:

$$ I_{G}(p) = 1 - \underset{i \in \{1,\ldots,k\}}\sum p_{i}^2.$$
Gini impurity values live on the half open interval $[0,1)$ where we are really looking for a value of $0$, which means the question we asked correctly classifies everything! 

In [77]:
import numpy as np
import pandas as pd
class Question:
    """A Question to label a node. """

    def __init__(self, feature, value, target):
        self.column = feature
        self.value = value
        self.target = target      

    def evaluate(self, data_frame):
        '''This function of the class breaks down how well the question separates classes of the target.'''
        
        #Initial impurity
        counts = np.unique(data_frame[self.target], return_counts = True)
        target_impurity = 1
        for i in range(len(counts[0])):
            probability_of_class = counts[1][i] / float(len(data_frame[self.target]))
            target_impurity -= probability_of_class**2
        self.initial_impurity = target_impurity
        
        
        # Compare the feature value in an example to the
        # feature value in this question.
        column_vals = data_frame[self.column]
        bool_list = []
        for row in column_vals: 
            if type(row) != str: 
                bool_list.append(row >= self.value)
                self.condition = '>='
            if type(row) == str:
                bool_list.append(row == self.value)
                self.condition = '='
        self.bool_list = bool_list
        
        #Now partition:
        true_rows = [i for i, val in enumerate(self.bool_list) if val] 
        false_rows = [i for i, val in enumerate(self.bool_list) if not val]
        self.true_data = data_frame.iloc[true_rows].reset_index(drop = True)
        self.false_data = data_frame.iloc[false_rows].reset_index(drop = True)
        
        #Counts the number of each type of example in a dataset.
        true_labels = [data_frame[self.target][i] for i in true_rows]
        false_labels = [data_frame[self.target][i] for i in false_rows]
        self.true_classes = true_labels
        self.false_classes = false_labels
        
        #Now calculate the gini impurity of each new node
        true_impurity = 1
        for i in range(len(np.unique(true_labels))):
            probability_of_class = np.unique(true_labels, return_counts = True)[1][i] / float(len(true_labels))
            true_impurity -= probability_of_class**2
        false_impurity = 1
        for i in range(len(np.unique(false_labels))):
            probability_of_class = np.unique(false_labels, return_counts = True)[1][i] / float(len(false_labels))
            false_impurity -= probability_of_class**2
        self.gini_impurity = np.array([true_impurity, false_impurity])   
        
        #Finally, we can evaluate the question as a whole:
        prob_true = len(true_labels)/(len(true_labels)+len(false_labels))
        probs = np.array([prob_true, 1-prob_true])
        self.evaluation = self.initial_impurity - np.sum(self.gini_impurity * probs) 
        return self

In [78]:
#Simple training set
sample_df = pd.DataFrame({'color': ['green', 'yellow', 'red','red','yellow'], 
                          'size': [3.0,3.0,1.0,1.0,3.0], 
                          'type': ['Apple','Apple', 'grape','grape','lemon'] })
#Now we can ask the question: Is the size greater than 2?
question1 = Question('size', 2, 'type')
question1.evaluate(sample_df)
question1.true_classes, question1.false_classes

(['Apple', 'Apple', 'lemon'], ['grape', 'grape'])

In [79]:
#It looks like this was a good start! Let's check it's gini impurity.
#It should have a perfect 0 for the false group
question1.gini_impurity

array([0.44444444, 0.        ])

In [80]:
#Let's evaluate the total question:
question1.evaluation

0.37333333333333324

In [81]:
#What about a categorical question?
#Is the color red?
question2 = Question('color', 'red', 'type')
question2.evaluate(sample_df)
question2.true_classes, question2.false_classes

(['grape', 'grape'], ['Apple', 'Apple', 'lemon'])

In [82]:
#It should have a perfect 0 for the true group
question2.gini_impurity

array([0.        , 0.44444444])

In [83]:
#Let's evaluate the total question:
question2.evaluation

0.37333333333333324

They're the same! Note that the question breaks down into two "child" branches of data.

In [84]:
question2.true_data

,color,size,type
0,red,1.0,grape
1,red,1.0,grape


In [85]:
question2.false_data

,color,size,type
0,green,3.0,Apple
1,yellow,3.0,Apple
2,yellow,3.0,lemon


In [86]:
question3 = Question('size', 10, 'type')

-------

### Evaluating questions
Now we have a way to evaluate how good a question is at splitting the data given a concrete question to "ask". So naturally, we need to abstract a little and find out how to evaluate a set of all possible questions-or at least a good amount of questions. 

In [89]:
def optimal_question(data_frame, target):
    '''Find the best question in a decision tree based on gini impurity'''
    #Question(feature, value, target)
    #question.evaluate(self, data_frame).evaluation
    
    columns = list(data_frame.columns)
    features = [val for i, val in enumerate(columns) if val != target ]
    data = data_frame[features]
    
    #We iterate through the columns and their values
    best_questions_per_feature = []
    for feature in list(data.columns):
        values = set(data[feature])
        what_question = []
        gini_imp = []
        for value in values:
            question = Question(feature, value, target)
            question.evaluate(data_frame)
            gini_imp.append(question.evaluation)
            what_question.append([feature, question.condition, value])
        index = [i for i, val in enumerate(gini_imp) if val == max(gini_imp)]
        best_questions_per_feature.append([what_question[index[0]], gini_imp[index[0]]])
    best_ginis = [x[1] for x in best_questions_per_feature]
    best_gini_index = [i for i, val in enumerate(best_ginis) if val == max(best_ginis)]   
    return best_questions_per_feature[best_gini_index[0]]

In [90]:
#For our simple example:
optimal_question(sample_df, 'type')

[['color', '=', 'red'], 0.37333333333333324]

In [136]:
class Leaf:
    """A Leaf node classifies data.
    """

    def __init__(self, data_frame,target):
        self.predictions = np.unique(data_frame[target], return_counts = True)
class Node:
    """This holds a reference to the question, and to the two child nodes for each node.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [169]:
def get_tree(data_frame, target):
    """Builds the decision tree recursively using the optimal question function.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    best_question = optimal_question(data_frame, target)[0]
    q_value = optimal_question(data_frame, target)[1]
    question = Question(best_question[0],best_question[2], target)
    question.evaluate(data_frame)
    
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if q_value == 0:
        return Leaf(data_frame, target)    
    
    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_data = question.true_data
    false_data = question.false_data

    # Recursively build the true branch.
    true_branch = get_tree(true_data, target)

    # Recursively build the false branch.
    false_branch = get_tree(false_data, target)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Node(best_question, true_branch, false_branch)

In [174]:
tree = get_tree(sample_df, 'type')

In [175]:
#This code I ripped from google developers that shows our decision tree. 
def show_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [176]:
print_tree(tree)

['color', '=', 'red']
--> True:
  Predict (array(['grape'], dtype=object), array([2]))
--> False:
  ['color', '=', 'yellow']
  --> True:
    Predict (array(['Apple', 'lemon'], dtype=object), array([1, 1]))
  --> False:
    Predict (array(['Apple'], dtype=object), array([1]))
